In [1]:
# import pandas as pd
#import numpy as np

import warnings
from functools import reduce
from itertools import chain
import pickle
import plotly.express as px
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
from CTV_functions import *
warnings.filterwarnings('ignore')

# 0. CARGA DE LA DATA

Se carga la data de las aplicaciones.

In [2]:
df_application = pd.read_csv("../data/application_record.csv")

Y también se carga la data de crédito.

In [3]:
df_credit = pd.read_csv('../data/credit_record.csv')  

Para esta parte se hace uso de las funciones contenidas en el script __*CTV_functions.py*__. Por favor, leer al final del script una importante __NOTA__ sobre las construcciones de estas funciones.

# 1. OBJETIVO.

Se parte de la suposición de que los datos crediticios corresponden al historial de comportamiento que muchos clientes tuvieron con respecto a un cuenta asociada a un producto de crédito.

La naturaleza de esta data permite observar, medir, y describir este comportamiento a través de un Análisis de Cohortes también conocido como Análsis Vintage o de la Cosecha.


El objetivo de esta tarea es aprovechar los resultados arrojados por el Análisis de Cohortes con el fin de definir una etiqueta de cliente "BUENO" o "MALO".

A su vez, esta etiqueta deberá contribuir en el desarrollo de una herramienta de Ciencia de Datos para predecir el perfil crediticio de los potenciales clientes de cierta firma.

Esta predicción se hará vía un modelo supervisado de clasificación que haga uso de esta etiqueta y de la data de aplicaciones.

# 2. EXPLORACIÓN.

Tener muy encuenta lo siguiente:
* __MONTH BALANCE__: El mes en el que extrajeron los datos es el punto de partida, hacia atrás.
    * 0 es el mes actual,
    * -1 es el mes anterior
    * ...
* __STATUS__:
    * __0:__ 1-29 días de atraso
    * __1:__ 30-59 días de atraso 
    * __2:__ 60-89 días de atraso 
    * __3:__ 90-119 días de atraso 
    * __4:__ 120-149 días de atraso 
    * __5:__ Atraso o deudas incobrables, cancelaciones por más de 150 días 
    * __C:__ Pagado ese mes 
    * __X:__ Sin préstamo durante el mes

In [4]:
df_credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [5]:
df_credit['ID'].value_counts()

5016769    61
5002806    61
5118192    61
5145767    61
5078567    61
           ..
5028795     1
5148442     1
5053900     1
5046332     1
5079047     1
Name: ID, Length: 45985, dtype: int64

Aunque la data tiene un tamaño de 1048575, en realidad la cantidad de usuarios es de 45985.

Se hace la siguiente reorganización de la información, de tal manera de ahora el `ID` se una columna.

In [6]:
df_pivot = df_credit.pivot(index='ID',
                           columns='MONTHS_BALANCE',
                           values='STATUS')

In [7]:
df_pivot

MONTHS_BALANCE,-60,-59,-58,-57,-56,-55,-54,-53,-52,-51,...,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
ID,,,,,,,,,,,,,,,,,,,,,
5001711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,X
5001712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,C,C,C,C,C,C,C,C,C
5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,X,X,X
5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,X,X,X
5001715,NaN,X,X,X,X,X,X,X,X,X,...,X,X,X,X,X,X,X,X,X,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5150483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,X,X,X
5150484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,C


Se puede ver que no todos los clientes tienen el mismo tamaño de información crediticia, es decir, el número de meses (__months on book__) en que ha estado activa una cuenta desde su apertura es distinto para cada usuario.

Esto, a su vez, da pie a varios tamaños de ventana (__window__) para el análisis, entiendiéndose esto como  la duración total en que una cuenta está activa, siendo el máximo uno de 60 meses.

Se calcula entonces lo siguiente, tomando en cuenta que se está trabajando con enteros negativos y cero:
* __opening_month__: El mes más pequeño, el cual se intepreta como el cual sucede la apertura de la cuenta.
* __closing_month__: El mes más grande, en el cual sucede el cierre de la cuenta.
* __window__: Ventana de análsis en la que cae el usuario.

In [8]:
df_pivot['opening_month'] = df_credit.groupby('ID')['MONTHS_BALANCE'].min()
df_pivot['closing_month'] = df_credit.groupby('ID')['MONTHS_BALANCE'].max()
df_pivot['ID'] = df_pivot.index
df_pivot

MONTHS_BALANCE,-60,-59,-58,-57,-56,-55,-54,-53,-52,-51,...,-6,-5,-4,-3,-2,-1,0,opening_month,closing_month,ID
ID,,,,,,,,,,,,,,,,,,,,,
5001711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,X,-3,0,5001711
5001712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,-18,0,5001712
5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,-21,0,5001713
5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,-14,0,5001714
5001715,NaN,X,X,X,X,X,X,X,X,X,...,X,X,X,X,X,X,X,-59,0,5001715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-28,-11,5150482
5150483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,X,X,X,X,X,X,X,-17,0,5150483
5150484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,C,-12,0,5150484


Por ahora, basta trabajar con la información de las últimas 3 columnas.

In [9]:
df_pivot_trunc = df_pivot[['ID', 'opening_month', 'closing_month']]

In [10]:
df_pivot_trunc['window'] = df_pivot_trunc['closing_month'] - df_pivot_trunc['opening_month']
df_pivot_trunc.reset_index(drop=True, inplace=True)

Se dan ejemplos de como luce la información de los clientes después de estos cálculos.

In [11]:
df_pivot_trunc.loc[45969:45981, :]

MONTHS_BALANCE,ID,opening_month,closing_month,window
45969,5150466,-1,0,1
45970,5150467,-52,-46,6
45971,5150468,-5,0,5
45972,5150473,-34,0,34
45973,5150475,-43,0,43
45974,5150476,-34,-13,21
45975,5150477,-20,0,20
45976,5150478,-13,0,13
45977,5150479,-8,0,8
45978,5150480,-49,-24,25


Se da una versión extendida de la data crediticia original.

In [12]:
df_credit_extend = pd.merge(df_credit,
                            df_pivot_trunc, 
                            on = 'ID',
                            how = 'left')

Se calcula el mes activo en el cual se da etiqueta `STATUS`.

In [13]:
df_credit_extend['month_on_book'] = df_credit_extend['MONTHS_BALANCE'] - df_credit_extend['opening_month']


In [14]:
df_credit_extend.shape

(1048575, 7)

 Y se da un ejemplo de como luce esta información para un usuario determinado.

In [15]:
df_credit_extend[df_credit_extend['ID']==5001711]

,ID,MONTHS_BALANCE,STATUS,opening_month,closing_month,window,month_on_book
0,5001711,0,X,-3,0,3,3
1,5001711,-1,0,-3,0,3,2
2,5001711,-2,0,-3,0,3,1
3,5001711,-3,0,-3,0,3,0


En este caso, el cliente con ID `5001711`, abrió su cuenta haces 3 meses con respecto al mes actual (`opening_month = -3`), su tamaño de información crediticia "completa" es de por supuesto 3 meses (`window = 3`) y se enlistan todos todos los meses en que la cuenta permaneció activa (`months on book`).

# 3. ELECCIÓN DE LOS TAMAÑOS DE VETANA A OBSERVAR.

Se calculan los porcentajes de clientes cuya cantidad de información total en meses (__window__) es más grande o igual a cierto umbral.

In [16]:
ratios = []
for i in range(0,61):
    ratio = df_pivot_trunc[df_pivot_trunc['window'] >= i].shape[0] / df_pivot_trunc.shape[0]
    ratios.append(ratio)

In [17]:
data = {'frecuencia de cuentas': ratios,
        'tamaño de ventana': list(range(0,61))}
df = pd.DataFrame(data)
df['frecuencia de cuentas'] = round(df['frecuencia de cuentas'] * 100, 2)

In [18]:
fig = px.scatter(df,
                 x='tamaño de ventana',
                 y='frecuencia de cuentas')
fig.add_trace(px.line(df,
                      x='tamaño de ventana',
                      y='frecuencia de cuentas').data[0])
fig.show()

La gráfica se interpreta como sigue:
* El porcentajes de clientes cuya ventana de información es mayor o igual a 1 es del 99.13 %.
* El porcentajes de clientes cuya ventana de información es mayor o igual a 10 es del 73.11 %.
* El porcentajes de clientes cuya ventana de información es mayor o igual a 12 es del 67.23 %.
* El porcentajes de clientes cuya ventana de información es mayor o igual a 18 es del 52.05 %.
* El porcentajes de clientes cuya ventana de información es mayor o igual a 20 es del 47.69 %.
* El porcentajes de clientes cuya ventana de información es mayor o igual a 30 es del 29.66 %.
* El porcentajes de clientes cuya ventana de información es mayor o igual a 60 es del 0.48 %.

Para este ejercicio, se considera interesente el trabajar con clientes cuyo tamaño de información crediticia no sea muy chica, digamos, como mínimo, tienen que tener 12 meses de información. Siendo así, todos aquellos clientes cuya información crediticia caiga en una ventana menor a 12 meses serán descartados. Aunque hay pérdida de información, aun se trabaja con el 67.23 % de los clientes.

# 4. ANÁLISIS DE COHORTES

Se definirán los siguientes __buquets__ de mora. Serán de tipo "acumulativos".

In [19]:
bucket_1 = ['1', '2', '3', '4', '5']  # Mora de 30 días o más.
bucket_2 = ['2', '3', '4', '5']  # Mora de 60 días o más.
bucket_3 = ['3', '4', '5']  # Mora de 90 días o más.
bucket_4 = ['4', '5']  # Mora de 120 días o más.
bucket_5 = ['5']  # Mora de 150 días o más.

Se eliminan los clientes que caen en ventanas con tamaños menores a 12. (__FILTRO 1__)

In [20]:
n = 12

Se hace el Análisis de Cohortes para cada bucket.

In [21]:
due_names = ['30 días o más',
             '60 días o más',
             '90 días o más',
             '120 días o más',
             '150 días o más']
dfs_vintage = []
all_buckets = [bucket_1, bucket_2, bucket_3, bucket_4, bucket_5]
iterator = iter(due_names)
for bucket in all_buckets:
    string = next(iterator)
    print(string)
    df_vintage, _ = get_vintage_analysis(n=n,
                                         df_credit_extend=df_credit_extend,
                                         chosen_bucket=bucket)
    dfs_vintage.append(df_vintage)

30 días o más
60 días o más
90 días o más
120 días o más
150 días o más


# 4.1 Un caso particular: 60 días o más

A fin de dar contexto acerca de cómo se interpretan las gŕaficas, se hará un análisis para un período de mora en particular, es decir __60 días o más__. En la subsección siguiente se explicará por qué este rango de mora es el más interesante para tener como riesgo a controlar.

Aquí, términos como "malos clientes" y "morosos" se usarán instistintamente.

In [22]:
df_vintage_2 = dfs_vintage[1]

In [23]:
example =  get_vintage_analysis_by_interval(df_vintage=df_vintage_2,
                                      time_interval=None,
                                      cohorts_list=[-54])
get_cohort_graph(example, bucket_2)

Se da contexto a un cohorte arbirtrario, digamos el de -54. Se tiene lo siguiente:
* En él están todos los clientes cuya cuenta se aperturó en el mes -54, es decir, hace 54 meses atrás después       de la extracción de la información. En este caso `opening_month=-54`.
* 10 meses depués de aperturada la cuenta (`month_on_book=10`), ya se tenía un porcentaje acumulado de 2.19 % morosos, es decir, clientes que tenían un atraso de 60 días o más.
* Después de 17 meses de apeturada la cuenta, se tiene un porcentaje acumulado de morosos de 3.65 %.
* La aparición de clientes morosos se da en los primeros 17 meses de haber aperturado la cuenta.
* No se detectan más morosos después del mes 17, se podría decir que la "curva se estabiliza".

Ahora, se dará un vistazo a todos los cohortes.

In [24]:
get_cohort_graph(df_vintage_2, bucket_2)

En general, parece que la mayor parte de morosos aparece en los primeros 15 (hacer un zoom abarcando hasta `month_on_book = 20`). A partir de ahí, las curvas de cohortes tienen a "estabilizarse".

Por otro lado, el acumulado máximo de morosos, que se da en el cohorte de -55, es de 5.22 %, lo cual ya deja ver un tremendo problema de desbalanceo entre el porcentaje de morosos y no morosos.

Si se hace zoom al principio de la gráfica, muy cerca del punto de nacimiento de las curvas y abarcando los primeros dos meses activos, también se natará la presencia de comportamientos extraños como, por ejemplo, que en los cohortes -53, -44, -38, -39, -12 ya se tienen clientes con moras de 60 o más días en el primer mes activo, lo cual no pareciera tener sentido. La data de agunos de estos clientes se ve así:

In [25]:
df_credit_extend_trunc = df_credit_extend[df_credit_extend['window'] >= n]
outliers = df_credit_extend_trunc[(df_credit_extend_trunc['opening_month'].isin([-53, -44, -38, -39, -12]))
                                   & (df_credit_extend_trunc['month_on_book']==1)
                                   & (df_credit_extend_trunc['STATUS'].isin(bucket_2))]['ID'].unique().tolist()
df_pivot.loc[outliers,-55:-37]

MONTHS_BALANCE,-55,-54,-53,-52,-51,-50,-49,-48,-47,-46,-45,-44,-43,-42,-41,-40,-39,-38,-37
ID,,,,,,,,,,,,,,,,,,,
5003267,NaN,NaN,1,2,3,3,1,1,C,C,C,C,C,C,C,C,C,C,C
5029311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5042064,NaN,NaN,0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
5056083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,2,X,2,0,C,C
5145817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5,0


Por ahora, no se tomará acción alguna sobre estos comportamientos extraños.

No existe un patrón de crecimiento de morosos tal como, por ejemplo, que mientras más antigua se la apertura, mayor probabilidad de observar morosos o viceversa. Como lo demuestran las siguientes curvas.

In [26]:
examples =  get_vintage_analysis_by_interval(df_vintage=df_vintage_2,
                                             time_interval=None,
                                             cohorts_list=[-12, -24, -15, -19, -20])
get_cohort_graph(examples, bucket_2)

En ellas, se puede ver que los acumulados de porcentajes de mora son superiores en los cohortes -19 y -15 con respecto a los de -12, a partir del mes activo 5, pero los acumulados para el cohorte de -12 son superiores en todo momento a los de los cohortes -20 y -24.

No se puede analizar cada una de las curvas a profundidad, pero sí pueden ser comparadas utilizando períodos de tiempo entre ellas dados, por ejemplo, por bimestre, trimestre, cuatrimeste, semestre y anual.
Haciendo un análsis previo de todas ellas, se elige hacer un comparativo cuatrimestral.

In [27]:
# POR CUATRIMESTRE
df_vintage_2_four = get_vintage_analysis_by_interval(df_vintage_2,
                                                     "four-month period")
get_cohort_graph(df_vintage_2_four, bucket_2)

Se puede observar que existen cohortes cuyos porcentajes de acumulados máximos de moros no rebasan el 1% (cohortes de -12, -20, -24), otro rebasan el 1% pero no el 2% (cohortes de -16, -28, -32). La mayoría de ellos tienen acumuludos máximos por arriba del 2%.

La anterior observación podría dar pie a un __filtro__ de información con el fin de dar realce a los morosos. Por ejemplo, se podría decir  que se dejará fuera de un posible análisis predictivo futuro a los integrantes de aquellos cohortes que no aporten nada o mímima información acerca de los morosos. 

Esto tiene sentido cuando se quiere una herramienta que tenga como prioridad el buen reconocimiento de los morosos.  Es cierto que se estará deshaciendo de parte de la información de los clientes buenos, pero se estará haciendo de tal manera que incurra minimamente en la perdida de información acerca de nuestro grupo de interés, el de los morosos.

Un ejemplo de filtro, podría ser el de dejar fuera a los clientes que integren los cohortes con un porcentaje acumulado máximo menor al 1% (__FILTRO_2__). Desde luego, esto es un valor arbitrario y que en la vida real, su asignación no debe caer en manos de una sola persona.

# 4.2. Cohortes medios.

Una manera de dar un vistazo rápido al comportamiento de los cohortes para los demás buckets de mora, es decir, 
de 30, de 90, de 120 y de 150 o más, y poder compararlos entre ellos es calculando el __cohorte medio__.

Se construye de la siguiente manera. Para cada __month_on_book__ se promedian las probabilidades acumuladas que se obtienen hasta ese momento. Lo anterior se hace para todos buckets. Se tendrán entonces 5 cohortes medios.

In [28]:
new_column_names = ['month_on_book'] + due_names
dfs_mean_cohort = []
# all_buckets = [bucket_1, bucket_2, bucket_3, bucket_4, bucket_5]
iterator = iter(new_column_names[1:])
for df in dfs_vintage:
    string = next(iterator)
    print(string)
    # df_vintage = get_vintage_analysis(n=n,
    #                                   df_credit_extend=df_credit_extend,
    #                                   chosen_bucket=bucket)
    df_mean_cohort = get_mean_cohort(df_vintage=df,
                                     chosen_bucket=bucket,
                                     with_graph=False)
    dfs_mean_cohort.append(df_mean_cohort)
combined_df = reduce(inner_join, dfs_mean_cohort)
combined_df.columns = new_column_names

30 días o más
60 días o más
90 días o más
120 días o más
150 días o más


In [29]:
df_long = combined_df.melt(id_vars='month_on_book',
                           var_name='months',
                           value_name='porcentajes')
fig = px.line(df_long,
              x='month_on_book',
              y='porcentajes',
              color='months',
              markers=True)
fig.show()

Se puede ver lo siguiente:
* Considerar un mora 30 o más, significa sobreestimar un riesgo que sucede muy habitualmente. Se considera entonces que esto es un evento no tan crítico. Por supuesto, el que algo sea "crítico" o no tiene que ir muy de acorde a los fines de la entidad interesada. Para fines pŕacticos, se considerará que no.

Haciendo un zoom en las curvas medias, dejando fuera a la de 30 días o más.:
* Hay una mayor probabilidad de observar un evento de 60 días de mora o más en todos los meses activos de la cuenta. De hecho, esta probabilidad se dispara a partir del segundo mes activo.
* La tasa de aparición de morosos crece mucho más lentamente en las moras que no son de 60 días o más.

Siendo así, es razonable darle prioridad a un evento cuya probabilidad aumenta de manera más acelarada como lo es __60 días o más de mora__.

# 5. CONSTRUCCIÓN DE LA VARIABLE OBJETIVO.

La etiqueta, desde luego, hará referencia a aquellos clientes que:
1. Tuvieron un atraso de 60 o más días. 
2. Tienen tamaño de ventana mayor o igual a 12 (__FILTRO_1__).
3. Que pertenezcan a cohortes cuyo porcentaje acumulado máximo de morosos sea mayor a 1% (__FILTRO_2__).

# 5.1 Pequeña exploración de la data de aplicaciones.

Se observa lo siguiente.

In [30]:
df_application.shape

(438557, 18)

In [31]:
count_ids = df_application['ID'].value_counts().to_frame().reset_index()
count_ids.columns = ['ID', 'count']
count_ids

,ID,count
0,7137299,2
1,7702238,2
2,7282535,2
3,7243768,2
4,7050948,2
...,...,...
438505,5690727,1
438506,6621262,1
438507,6621261,1
438508,6621260,1


In [32]:
count_ids[count_ids['count'] >= 2].shape

(47, 2)

Existen 47 casos de repeticion de `ID`'s.Se necesita revisar por qué sucede esto. Por ende, estos `ID`'s no se considerarán en la versión final de la data de aplicaciones.

In [33]:
unconsidered_ids = count_ids[count_ids['count'] >= 2]['ID']
df_application = df_application[-df_application['ID'].isin(unconsidered_ids)]
df_application.shape

(438463, 18)

# 5.2 Aplicación de Filtros.

Se obtiene el Análsis de Cohortes así como la ID's de malos clientes que caen en ellos.

In [34]:
df_vintage, due_ids_in_cohorts = get_vintage_analysis(n=n,
                                                      df_credit_extend=df_credit_extend,
                                                      chosen_bucket=bucket_2)

Se localizan a los morosos en cada cohorte considerado:

In [35]:
bad_ids_set = set(chain(*due_ids_in_cohorts))
bad_ids = list(bad_ids_set)

Se localizan los cohortes que cumplen el __FILTRO_2__.

In [36]:
df_vintage_pivot = df_vintage.pivot(index='opening_month',
                                    columns='month_on_book',
                                    values='due_rate').reset_index()
df_vintage_pivot['True_umbral'] = df_vintage_pivot.apply(lambda row: any(val >= 1 for val in row[1:]),
                                                         axis=1)
cohortes_True_list = list(df_vintage_pivot[df_vintage_pivot['True_umbral']==True]['opening_month'])

Se aplican los __FILTRO 1__ y __FILTRO_2__ a la data crediticia.

In [37]:
df_credit_extend_trunc = df_credit_extend[(df_credit_extend['window'] >= n) 
                                          & (df_credit_extend['opening_month'].isin(cohortes_True_list))]

Se localizan el total de ids involucrados y se obtiene la data de aplicación final.

In [38]:
total_ids = list(df_credit_extend_trunc['ID'].unique())
df_application_final = df_application[df_application['ID'].isin(total_ids)]
df_application_final['STATUS'] = 0

Se agrega la etiquete `STATUS` a la data de aplicación. El valor 1 quiere decir un "MAL" cliente y el valor 0 es un "BUEN" cliente.

In [39]:
bad_aplicants_index_list = (df_application_final['ID'].isin(bad_ids))
df_application_final.loc[bad_aplicants_index_list, 'STATUS'] = 1

In [40]:
df_application_final.reset_index(drop=True, inplace=True)

# 5.3 Configuración de la variable objetivo.

La variable objetivo tiene la siguiente configuración.

In [41]:
count_objected = df_application_final['STATUS'].value_counts().to_frame().reset_index()
count_objected.columns = ['STATUS', 'count']
count_objected

,STATUS,count
0,0,19313
1,1,499


In [42]:
fig = px.pie(count_objected,
             values='count',
             names='STATUS',
             title='MALO o BUENO')
fig.show()

Hay un desbalanceo importante entre las clases.

# 5.4 Versión alternativa de la data de aplicaciones.

Para fines prácticos, se dará un mayor realce de la clase minoritaria haciendo un submuestreo arbitrario de la clase mayoritaria. Esto se hará de tal manera que la clase minoritaria ahora represente el 10% de la data total.

In [43]:
(499 * 90) / 10

4491.0

In [44]:
STATUS_1 = df_application_final.loc[df_application_final['STATUS']==1,:]
STATUS_0 = df_application_final.loc[df_application_final['STATUS']==0,:]
STATUS_0_new = STATUS_0.sample(n=4491) 
frames = [STATUS_1, STATUS_0_new]
df_application_final_new = pd.concat(frames)
df_application_final_new.reset_index(drop=True, inplace=True)

In [45]:
df_application_final_new

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,5009628,F,N,N,0,238500.0,Working,Secondary / secondary special,Married,House / apartment,-19305,-3296,1,1,0,0,Laborers,2.0,1
1,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557,-586,1,1,1,0,NaN,2.0,1
2,5009938,F,N,Y,2,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10710,-2351,1,0,0,0,Sales staff,4.0,1
3,5010061,F,N,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-16573,-4574,1,0,0,0,NaN,2.0,1
4,5010535,F,Y,N,2,135000.0,Working,Secondary / secondary special,Married,House / apartment,-13928,-3052,1,0,0,0,Laborers,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,5052619,M,Y,Y,0,180000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-18475,-1782,1,0,0,0,Cleaning staff,1.0,0
4986,5053239,M,Y,Y,2,270000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-14477,-293,1,1,1,0,Drivers,4.0,0
4987,5024773,F,Y,Y,0,202500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-8568,-1068,1,0,0,0,Sales staff,1.0,0
4988,5085963,F,N,Y,0,180000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-18913,-807,1,0,0,0,Laborers,1.0,0


In [46]:
count_objected = df_application_final_new['STATUS'].value_counts().to_frame().reset_index()
count_objected.columns = ['STATUS', 'count']
fig = px.pie(count_objected,
             values='count',
             names='STATUS',
             title='MALO o BUENO')
fig.show()

In [47]:
df_application_final_new

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,5009628,F,N,N,0,238500.0,Working,Secondary / secondary special,Married,House / apartment,-19305,-3296,1,1,0,0,Laborers,2.0,1
1,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557,-586,1,1,1,0,NaN,2.0,1
2,5009938,F,N,Y,2,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10710,-2351,1,0,0,0,Sales staff,4.0,1
3,5010061,F,N,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-16573,-4574,1,0,0,0,NaN,2.0,1
4,5010535,F,Y,N,2,135000.0,Working,Secondary / secondary special,Married,House / apartment,-13928,-3052,1,0,0,0,Laborers,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,5052619,M,Y,Y,0,180000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-18475,-1782,1,0,0,0,Cleaning staff,1.0,0
4986,5053239,M,Y,Y,2,270000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-14477,-293,1,1,1,0,Drivers,4.0,0
4987,5024773,F,Y,Y,0,202500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-8568,-1068,1,0,0,0,Sales staff,1.0,0
4988,5085963,F,N,Y,0,180000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-18913,-807,1,0,0,0,Laborers,1.0,0


# 6. SOBRE LA HERRAMIENTA DE CIENCIA DE DATOS.

Se supondrá que para el desarrollo de la herramienta que prediga el perfil crediticio solo se deberá de hacer uso de la data de aplicaciones. Por tanto, no se usará la información crediticia derivada de este Análisis de Cohortes para desarrollar algún tipo de ingeniería de características con ella.

# 7. GUARDADO DE LA DATA

In [ ]:
# pickle.dump(df_application_final, open(f'../outputs/df_application_final.sav', 'wb'))
# pickle.dump(df_application_final_new, open(f'../outputs/df_application_final_new.sav', 'wb'))